In [5]:
pip install selenium beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

def clicca_mostra_altro(driver, max_clicks=20, pausa=5):
    """
    Clicca ripetutamente sul pulsante 'Mostra altro' (o simile),
    fino a esaurimento o max_clicks tentativi.
    """
    for i in range(max_clicks):
        try:
            # Aggiorna il selettore se il bottone cambia nome o classe
            bottone = driver.find_element(By.CSS_SELECTOR, 'button.show-more-button')
            if bottone.is_displayed() and bottone.is_enabled():
                print(f"Clicco sul pulsante Mostra altro ({i+1}/{max_clicks})...")
                bottone.click()
                time.sleep(pausa)
            else:
                print("Pulsante Mostra altro non cliccabile.")
                break
        except NoSuchElementException:
            print("Pulsante Mostra altro non trovato, termino il caricamento.")
            break
        except (ElementClickInterceptedException, StaleElementReferenceException):
            # Se bottone scompare o blocco temporaneo, aspetta e riprova
            print("Errore cliccando pulsante, aspetto e riprovo...")
            time.sleep(pausa)
    print("Fine caricamento con Mostra altro.")

def estrai_opere_guggenheim(url, output_csv):
    chrome_options = Options()
    chrome_options.add_argument("--headless")           # modalità headless (senza GUI)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)

    try:
        print(f"Apro la pagina: {url}")
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # Aspetta che il primo artista venga caricato per sicurezza
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ArtworkPreview-artist")))
        print("Pagina caricata, inizio a caricare tutte le opere...")

        # Clicca tutti i "Mostra altro" per caricare la lista completa
        clicca_mostra_altro(driver, max_clicks=20, pausa=5)

        # Dopo caricamento finale, aspetta ancora i dati caricati
        time.sleep(3)

        # Rileva tutti i campi
        artisti = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-artist")
        titoli = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-title")
        date = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-date")
        stati = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-status")

        # Valuta il numero di opere presenti considerando il minimo tra le liste
        num_opere = min(len(artisti), len(titoli), len(date), len(stati))
        print(f"Trovate {num_opere} opere (basato sul minimo comune tra campi)")

        opere = []

        for i in range(num_opere):
            artista = artisti[i].text.strip()
            titolo = titoli[i].text.strip()
            anno = date[i].text.strip()
            stato = stati[i].text.strip()

            opere.append({
                'Artista': artista,
                'Titolo': titolo,
                'Anno': anno,
                'Stato': stato
            })
            print(f"{i+1}: Artista: {artista} | Titolo: {titolo} | Anno: {anno} | Stato: {stato}")

        # Salvataggio CSV
        with open(output_csv, 'w', encoding='utf-8', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=['Artista', 'Titolo', 'Anno', 'Stato'])
            writer.writeheader()
            for opera in opere:
                writer.writerow(opera)

        print(f"Dati salvati in '{output_csv}'")

    finally:
        driver.quit()

if __name__ == "__main__":
    url = "https://www.guggenheim-venice.it/it/arte/opere/"
    output_csv = "guggenheim_opere_completo.csv"
    estrai_opere_guggenheim(url, output_csv)


Apro la pagina: https://www.guggenheim-venice.it/it/arte/opere/
Pagina caricata, inizio a caricare tutte le opere...
Pulsante Mostra altro non trovato, termino il caricamento.
Fine caricamento con Mostra altro.
Trovate 24 opere (basato sul minimo comune tra campi)
1: Artista: RENÉ MAGRITTE | Titolo: L’impero della luce | Anno: 1953-54 | Stato: OPERA ESPOSTA
2: Artista: SALVADOR DALÍ | Titolo: La nascita dei desideri liquidi | Anno: 1931-32 | Stato: OPERA ESPOSTA
3: Artista:  | Titolo:  | Anno:  | Stato: 
4: Artista:  | Titolo:  | Anno:  | Stato: 
5: Artista:  | Titolo:  | Anno:  | Stato: 
6: Artista:  | Titolo:  | Anno:  | Stato: 
7: Artista:  | Titolo:  | Anno:  | Stato: 
8: Artista:  | Titolo:  | Anno:  | Stato: 
9: Artista:  | Titolo:  | Anno:  | Stato: 
10: Artista:  | Titolo:  | Anno:  | Stato: 
11: Artista:  | Titolo:  | Anno:  | Stato: 
12: Artista:  | Titolo:  | Anno:  | Stato: 
13: Artista:  | Titolo:  | Anno:  | Stato: 
14: Artista:  | Titolo:  | Anno:  | Stato: 
15: Artista:

In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

def scroll_infinito(driver, pausa=3, max_scroll=30):
    """Scorre la pagina verso il basso per caricare tutti i contenuti dinamici."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(max_scroll):
        print(f"Scroll {i+1}/{max_scroll}...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pausa)  # attesa per caricamento contenuti

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Fine del caricamento: altezza pagina non cambia più.")
            break
        last_height = new_height

def estrai_dati_opere(driver):
    """Estrae i dati delle opere usando i selettori CSS forniti."""
    artisti = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-artist")
    titoli = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-title")
    date = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-date")
    stati = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-status")

    num_opere = min(len(artisti), len(titoli), len(date), len(stati))
    print(f"Trovate {num_opere} opere (basato sul minimo tra i campi)")

    opere = []
    for i in range(num_opere):
        artista = artisti[i].text.strip()
        titolo = titoli[i].text.strip()
        anno = date[i].text.strip()
        stato = stati[i].text.strip()

        opere.append({
            'Artista': artista,
            'Titolo': titolo,
            'Anno': anno,
            'Stato': stato
        })

        print(f"{i+1}: Artista: {artista} | Titolo: {titolo} | Anno: {anno} | Stato: {stato}")

    return opere

def salva_csv(opere, filename):
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Artista', 'Titolo', 'Anno', 'Stato'])
        writer.writeheader()
        for opera in opere:
            writer.writerow(opera)
    print(f"Dati salvati in '{filename}'")

def main():
    url = "https://www.guggenheim-venice.it/it/arte/opere/"
    output_csv = "guggenheim_opere_completo.csv"

    chrome_options = Options()
    chrome_options.add_argument("--headless")  # senza GUI
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 20)

    try:
        print(f"Apro la pagina: {url}")
        driver.get(url)

        # Aspetto che almeno un elemento artista sia presente
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ArtworkPreview-artist")))
        print("Pagina caricata correttamente.")

        # Scorro la pagina per caricare tutti i dati dinamici
        scroll_infinito(driver, pausa=3, max_scroll=30)

        # Attendo che gli elementi siano ancora presenti dopo scroll
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ArtworkPreview-artist")))

        # Ora estraggo i dati
        opere = estrai_dati_opere(driver)

        # Salvo i dati su CSV
        salva_csv(opere, output_csv)

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


Apro la pagina: https://www.guggenheim-venice.it/it/arte/opere/
Pagina caricata correttamente.
Scroll 1/30...
Scroll 2/30...
Fine del caricamento: altezza pagina non cambia più.
Trovate 24 opere (basato sul minimo tra i campi)
1: Artista: RENÉ MAGRITTE | Titolo: L’impero della luce | Anno: 1953-54 | Stato: OPERA ESPOSTA
2: Artista: SALVADOR DALÍ | Titolo: La nascita dei desideri liquidi | Anno: 1931-32 | Stato: OPERA ESPOSTA
3: Artista: VASILY KANDINSKY | Titolo: Verso l'alto | Anno: 1929 | Stato: OPERA ESPOSTA
4: Artista: PAUL KLEE | Titolo: Ritratto di Frau P. nel Sud | Anno: 1924 | Stato: OPERA ESPOSTA
5: Artista: PABLO PICASSO | Titolo: Sulla spiaggia | Anno: 1937 | Stato: OPERA NON ESPOSTA
6: Artista: PABLO PICASSO | Titolo: Lo studio | Anno: 1928 | Stato: OPERA ESPOSTA
7: Artista: JACKSON POLLOCK | Titolo: La donna luna | Anno: 1942 | Stato: OPERA ESPOSTA
8: Artista: JACKSON POLLOCK | Titolo: Foresta incantata | Anno: 1947 | Stato: OPERA ESPOSTA
9: Artista: GIORGIO DE CHIRICO | T

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

def estrai_dati_opere(driver):
    artisti = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-artist")
    titoli = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-title")
    date = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-date")
    stati = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-status")

    num_opere = min(len(artisti), len(titoli), len(date), len(stati))
    print(f"Trovate {num_opere} opere in questa pagina.")

    opere = []
    for i in range(num_opere):
        artista = artisti[i].text.strip()
        titolo = titoli[i].text.strip()
        anno = date[i].text.strip()
        stato = stati[i].text.strip()

        opere.append({
            'Artista': artista,
            'Titolo': titolo,
            'Anno': anno,
            'Stato': stato
        })
        print(f"  {i+1}: {artista} - {titolo} - {anno} - {stato}")
    return opere

def salva_csv(opere, filename):
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Artista', 'Titolo', 'Anno', 'Stato'])
        writer.writeheader()
        for opera in opere:
            writer.writerow(opera)
    print(f"Tutti i dati salvati in '{filename}'")

def main():
    base_url = "https://www.guggenheim-venice.it/it/arte/opere/"
    # Pagine da 1 (index base url) a 26
    pagina_inizio = 1
    pagina_fine = 26
    output_csv = "guggenheim_opere_tutte_le_pagine.csv"

    chrome_options = Options()
    chrome_options.add_argument("--headless")  # modalità headless (senza GUI)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 20)

    tutte_le_opere = []

    try:
        for pagina in range(pagina_inizio, pagina_fine + 1):
            if pagina == 1:
                url = base_url
            else:
                url = f"{base_url}page{pagina}"

            print(f"\n--- Apertura pagina {pagina}: {url}")
            driver.get(url)

            # Aspettiamo che la pagina sia caricata (almeno 1 artista)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ArtworkPreview-artist")))

            # Breve pausa per completezza caricamento
            time.sleep(2)

            # Estrai opere da pagina corrente
            opere_pagina = estrai_dati_opere(driver)
            tutte_le_opere.extend(opere_pagina)

        # Finito il ciclo scrivo tutto su CSV
        salva_csv(tutte_le_opere, output_csv)

    finally:
        driver.quit()

if __name__ == "__main__":
    main()



--- Apertura pagina 1: https://www.guggenheim-venice.it/it/arte/opere/
Trovate 24 opere in questa pagina.
  1: RENÉ MAGRITTE - L’impero della luce - 1953-54 - OPERA ESPOSTA
  2: SALVADOR DALÍ - La nascita dei desideri liquidi - 1931-32 - OPERA ESPOSTA
  3:  -  -  - 
  4:  -  -  - 
  5:  -  -  - 
  6:  -  -  - 
  7:  -  -  - 
  8:  -  -  - 
  9:  -  -  - 
  10:  -  -  - 
  11:  -  -  - 
  12:  -  -  - 
  13:  -  -  - 
  14:  -  -  - 
  15:  -  -  - 
  16:  -  -  - 
  17:  -  -  - 
  18:  -  -  - 
  19:  -  -  - 
  20:  -  -  - 
  21:  -  -  - 
  22:  -  -  - 
  23:  -  -  - 
  24:  -  -  - 

--- Apertura pagina 2: https://www.guggenheim-venice.it/it/arte/opere/page2
Trovate 24 opere in questa pagina.
  1: CARLA ACCARDI - Blu concentrico - 1960 - OPERA NON ESPOSTA
  2: CARLA ACCARDI - Senza titolo - FINE ANNI '60 - OPERA NON ESPOSTA
  3:  -  -  - 
  4:  -  -  - 
  5:  -  -  - 
  6:  -  -  - 
  7:  -  -  - 
  8:  -  -  - 
  9:  -  -  - 
  10:  -  -  - 
  11:  -  -  - 
  12:  -  -  - 
  1

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

def scroll_infinito(driver, pausa=2, max_scroll=30):
    """Scorre la pagina verso il basso più volte per caricare contenuti dinamici."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(max_scroll):
        print(f"- Scroll {i+1}/{max_scroll}...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pausa)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("  Fine caricamento: altezza pagina stabile.")
            break
        last_height = new_height

def estrai_dati_opere(driver):
    artisti = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-artist")
    titoli = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-title")
    date = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-date")
    stati = driver.find_elements(By.CLASS_NAME, "ArtworkPreview-status")

    num_opere = min(len(artisti), len(titoli), len(date), len(stati))
    print(f"Trovate {num_opere} opere in questa pagina dopo scroll.")

    opere = []
    for i in range(num_opere):
        artista = artisti[i].text.strip()
        titolo = titoli[i].text.strip()
        anno = date[i].text.strip()
        stato = stati[i].text.strip()

        opere.append({
            'Artista': artista,
            'Titolo': titolo,
            'Anno': anno,
            'Stato': stato
        })
        print(f"  {i+1}: {artista} | {titolo} | {anno} | {stato}")

    return opere

def salva_csv(opere, filename):
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['Artista', 'Titolo', 'Anno', 'Stato'])
        writer.writeheader()
        for opera in opere:
            writer.writerow(opera)
    print(f"\nTutti i dati salvati in '{filename}'")

def main():
    base_url = "https://www.guggenheim-venice.it/it/arte/opere/"
    pagina_inizio = 1
    pagina_fine = 26
    output_csv = "guggenheim_opere_tutte_le_pagine.csv"

    chrome_options = Options()
    chrome_options.add_argument("--headless")     # per esecuzione senza GUI
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 20)

    tutte_le_opere = []

    try:
        for pagina in range(pagina_inizio, pagina_fine + 1):
            if pagina == 1:
                url = base_url
            else:
                url = f"{base_url}page{pagina}"

            print(f"\n=== Apertura pagina {pagina}: {url}")
            driver.get(url)

            # Aspetto che almeno un artista sia presente sulla pagina
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ArtworkPreview-artist")))

            # Scroll infinito per caricare TUTTE le opere della pagina
            scroll_infinito(driver, pausa=2, max_scroll=20)

            # Estraggo le opere dopo completezza caricamento
            opere_pagina = estrai_dati_opere(driver)
            tutte_le_opere.extend(opere_pagina)

        # Salvo tutto in unico CSV dopo aver girato tutte le pagine
        salva_csv(tutte_le_opere, output_csv)

    finally:
        driver.quit()

if __name__ == "__main__":
    main()



=== Apertura pagina 1: https://www.guggenheim-venice.it/it/arte/opere/
- Scroll 1/20...
- Scroll 2/20...
  Fine caricamento: altezza pagina stabile.
Trovate 24 opere in questa pagina dopo scroll.
  1: RENÉ MAGRITTE | L’impero della luce | 1953-54 | OPERA ESPOSTA
  2: SALVADOR DALÍ | La nascita dei desideri liquidi | 1931-32 | OPERA ESPOSTA
  3: VASILY KANDINSKY | Verso l'alto | 1929 | OPERA ESPOSTA
  4: PAUL KLEE | Ritratto di Frau P. nel Sud | 1924 | OPERA ESPOSTA
  5: PABLO PICASSO | Sulla spiaggia | 1937 | OPERA NON ESPOSTA
  6: PABLO PICASSO | Lo studio | 1928 | OPERA ESPOSTA
  7: JACKSON POLLOCK | La donna luna | 1942 | OPERA ESPOSTA
  8: JACKSON POLLOCK | Foresta incantata | 1947 | OPERA ESPOSTA
  9: GIORGIO DE CHIRICO | La torre rossa | 1913 | OPERA NON ESPOSTA
  10: MAX ERNST | La vestizione della sposa | 1940 | OPERA ESPOSTA
  11: CONSTANTIN BRANCUSI | Maiastra | 1912 C. | OPERA ESPOSTA
  12: KAZIMIR MALEVICH | Senza titolo | 1916 C. | OPERA ESPOSTA
  13: GINO SEVERINI | Mare